# Confusion Matrix - MLP (AE Min 8 Node)

In [1]:
import numpy as np
import pickle
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def one_hot_convert_normal(data, end_line, output_num):
    list_data = []

    for row in range(0, end_line):
        for column in range(0, output_num):
            if data[row][column] == 1:
                list_data.append(column + 1)
    return list_data

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

In [6]:
# Training data
output_num = 10
end_line = train_set_num * output_num

oh_y_in_train_denoised = one_hot_convert_normal(y_in_train_denoised, end_line, output_num)
oh_y_in_train_denoised = np.array(oh_y_in_train_denoised)

oh_y_out_train_denoised = one_hot_convert_normal(y_out_train_denoised, end_line, output_num)
oh_y_out_train_denoised = np.array(oh_y_out_train_denoised)

# Testing data
end_line = test_set_num * output_num

oh_y_in_test_denoised = one_hot_convert_normal(y_in_test_denoised, end_line, output_num)
oh_y_in_test_denoised = np.array(oh_y_in_test_denoised)

oh_y_out_test_denoised = one_hot_convert_normal(y_out_test_denoised, end_line, output_num)
oh_y_out_test_denoised = np.array(oh_y_out_test_denoised)

In [7]:
autoencoder_01 = load_model('autoencoder_01.h5')
autoencoder_02 = load_model('autoencoder_02.h5')
autoencoder_03 = load_model('autoencoder_03.h5')
autoencoder_04 = load_model('autoencoder_04.h5')

# [ 방법 1 ]
## 데이터 10개의 평균 사용

In [8]:
import numpy as np

def average(x, y, set_num, data_num):
    idx = 0
    sum = np.array([0, 0, 0])
    data = [[0 for col in range(3)] for row in range(int(set_num * 10 / data_num))]
    label = [[0 for col in range(10)] for row in range(int(set_num * 10 / data_num))]
    for i in range(set_num * 10):
        for j in range(3):
            sum[j] += x[i][j]
        if(i % data_num == data_num - 1):
            for j in range(3):
                data[idx][j] = int(round(sum[j] / data_num))
            for j in range(10):
                label[idx][j] = y[i][j]
            idx += 1
            sum = np.array([0, 0, 0])    
    return data, label

In [9]:
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy) * (-100)
x_train, y_train = average(after_autoencoder_x_train, y_in_train_noisy, 4500, 10)

In [10]:
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy) * (-100)
x_test, y_test = average(after_autoencoder_x_test, y_in_test_noisy, 1500, 10)

In [11]:
x_train = np.array(x_train)
x_test = np.array(x_test)
ae_x_in_train = x_train.astype('float32') / -100.
ae_x_in_test = x_test.astype('float32') / -100.

In [12]:
# Average data
# Train data
oh_y_in_train = one_hot_convert_normal(y_train, int(4500 * output_num / 10), output_num)
oh_y_in_train = np.array(oh_y_in_train)

# Test data
oh_y_in_test = one_hot_convert_normal(y_test, int(1500 * output_num / 10), output_num)
oh_y_in_test = np.array(oh_y_in_test)

## MLP 01

In [13]:
mlp_01 = pickle.load(open('mlp_01.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = mlp_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = mlp_01.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = mlp_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = mlp_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = mlp_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = mlp_01.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_01.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[   0 4472    0    0    0    0    0    0    0   28]
 [   0 4357    0    0    8    0    0   18    0  117]
 [   0 4500    0    0    0    0    0    0    0    0]
 [   0 4500    0    0    0    0    0    0    0    0]
 [   0 4500    0    0    0    0    0    0    0    0]
 [   0 4500    0    0    0    0    0    0    0    0]
 [   0   76  603    0  157 1560   71 1519    0  514]
 [4486    0    0    0    0    0    0    5    0    9]
 [   0 4496    0    0    0    0    0    0    0    4]
 [   0 4500    0    0    0    0    0    0    0    0]]


Testing data Confusion Matrix
[[   0 1492    0    0    0    0    0    0    0    8]
 [   0 1450    0    0    6    0    0    5    0   39]
 [   0 1500    0    0    0    0    0    0    0    0]
 [   0 1500    0    0    0    0    0    0    0    0]
 [   0 1500    0    0    0    0    0    0    0    0]
 [   0 1500    0    0    0    0    0    0    0    0]
 [   0   28  199    0   52  477   24  529    0  191]
 [1492    0    0    0    0

## MLP 02

In [14]:
mlp_02 = pickle.load(open('mlp_02.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = mlp_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = mlp_02.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = mlp_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = mlp_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = mlp_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = mlp_02.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_02.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[4392   70    0    0    0    0   28    0    0   10]
 [ 124 3930    5    0    0    0  149    0    0  292]
 [   0    0 4492    0    0    0    0    0    8    0]
 [ 169    5    0 4186  122    0    0    0   18    0]
 [   0    0    0   46 4149   19    0    0  286    0]
 [   0    0    0    0    0 4312    0    0  188    0]
 [   1   50    0    0    0    0 4441    0    7    1]
 [   0    0    0    0    0    0   32 4468    0    0]
 [   0    0    4   69  236  191    5    0 3995    0]
 [   0  176    0    0    0    0    0    0    0 4324]]


Testing data Confusion Matrix
[[1469   20    0    0    0    0    8    0    0    3]
 [  44 1287    4    0    0    0   52    0    0  113]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  61    0    1 1388   43    0    0    0    7    0]
 [   0    0    0   12 1384    3    0    0  101    0]
 [   0    0    0    0    0 1447    0    0   53    0]
 [   0   17    0    0    0    0 1479    0    3    1]
 [   0    0    0    0    0

## MLP 03

In [15]:
mlp_03 = pickle.load(open('mlp_03.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = mlp_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = mlp_03.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = mlp_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = mlp_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = mlp_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = mlp_03.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_03.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[4230  203    0    0    0    0   39    0    0   28]
 [ 123 3917    6    0    0    0  160    0    0  294]
 [   0    0 4492    0    0    0    0    0    8    0]
 [  99   74    1 4118  192    0    0    0   16    0]
 [   0    0    0   39 4215   32    0    0  214    0]
 [   0    0    0    0    8 4338    0    0  154    0]
 [   0   45    0    0    0    0 4436    0   13    6]
 [   0    0    0    0    0    0   28 4472    0    0]
 [   0    0    4   44  310  249    4    0 3889    0]
 [   0  169    0    0    0    0    0    0    0 4331]]


Testing data Confusion Matrix
[[1415   68    0    0    0    0   10    0    0    7]
 [  46 1279    4    0    0    0   58    0    0  113]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  38   23    1 1367   65    0    0    0    6    0]
 [   0    0    0   12 1412    8    0    0   68    0]
 [   0    0    0    0    2 1455    0    0   43    0]
 [   0   17    0    0    0    0 1477    0    5    1]
 [   0    0    0    0    0

## MLP 04

In [16]:
mlp_04 = pickle.load(open('mlp_04.sav', 'rb'))

print("test 01\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

prediction = mlp_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 02\n")
prediction = mlp_04.predict(x_out_train_denoised)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(x_out_test_denoised)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 03\n")
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

prediction = mlp_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 04\n")
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_03.predict(ae_x_in_test_noisy) * (-100)

prediction = mlp_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")


print("test 05\n")
after_autoencoder_x_train = autoencoder_01.predict(ae_x_in_train_noisy)
after_autoencoder_x_test = autoencoder_01.predict(ae_x_in_test_noisy)
after_autoencoder_x_train = autoencoder_04.predict(after_autoencoder_x_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(after_autoencoder_x_test) * (-100)

prediction = mlp_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_out_train_denoised, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_out_test_denoised, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n---\n")

print("test 06\n")
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test) * (-100)

prediction = mlp_04.predict(after_autoencoder_x_train)
cm = confusion_matrix(oh_y_in_train, prediction)
print("Training data Confusion Matrix")
print(cm)
print("\n")

prediction = mlp_04.predict(after_autoencoder_x_test)
cm = confusion_matrix(oh_y_in_test, prediction)
print("Testing data Confusion Matrix")
print(cm)
print("\n")

test 01

Training data Confusion Matrix
[[4264  169    0    0    0    0   14    0    7   46]
 [ 130 3939   10    0    0    0  111    0   41  269]
 [   0    0 4492    0    0    0    0    0    8    0]
 [ 101   72    1 3988  313    0    0    0   25    0]
 [   0    0    0  339 3943   19    0    0  199    0]
 [   0    0    0    0    8 4362    0    0  130    0]
 [   0   45    0    0    0    0 4353    0   57   45]
 [   0    0    0    0    0    0   32 4468    0    0]
 [   0    0    4  245  165  263    4    0 3819    0]
 [   0  211    0    0    0    0    0    0    0 4289]]


Testing data Confusion Matrix
[[1423   60    0    0    0    0    6    0    2    9]
 [  48 1295    5    0    0    0   44    0   12   96]
 [   0    0 1498    0    0    0    0    0    2    0]
 [  38   23    1 1324  107    0    0    0    7    0]
 [   0    0    0  124 1312    3    0    0   61    0]
 [   0    0    0    0    2 1462    0    0   36    0]
 [   0   17    0    0    0    0 1451    0   20   12]
 [   0    0    0    0    0